In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [2]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [3]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
#     print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [4]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
#         print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
#         print 'values',values
#         print 'branchedNames',branchedNames
#         print 'branched',branched

        if len(values)>0:  
            BranchedMatrix.append([branched,values])
#             BranchedValue.append(values)
#             print 'BranchedMatrix',BranchedMatrix

            if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
                SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
                SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
#                 print 'SecondModelOutput[1]',SecondModelOutput[1]
#                 print 'SecondModelOutput[0]',SecondModelOutput[0]
                if SecondModelOutput[1]>0.000001:
    #                 print 'bayad cut bezane'
                    output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
#                     print 'output',output[0]
#                     print 'output[1]',output[1]
                    lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
#                     print 'lhs',lhs[0]
                    self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
                    print 'cut added'



In [5]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))

# admipex3(c)
CutCplex(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Cuts_Cliques                        -1
CPXPARAM_MIP_Cuts_Covers                         -1
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Cuts_FlowCovers                     -1
CPXPARAM_MIP_Cuts_Implied                        -1
CPXPARAM_MIP_Cuts_GUBCovers                      -1
CPXPARAM_MIP_Cuts_Gomory                         -1
CPXPARAM_MIP_Cuts_PathCut                        -1
CPXPARAM_MIP_Cuts_MIRCut                         -1
CPXPARAM_MIP_Cuts_Disjunctive                    -1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_MIP_Cuts_ZeroHalfCut                    -1
CPXPARAM_MIP_Cuts_MCFCut                         -1
CPXPARAM_MIP_Cuts_LiftProj                       -1
CPXPARAM_MIP_Cuts_LocalImplied                   -1
CPXPARAM_MIP_Cuts_BQP                            -1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 30 rows, 30 col

Iteration:     1   Scaled dual infeas =          4273.386384
Iteration:    68   Scaled dual infeas =           281.457491
Iteration:   131   Scaled dual infeas =           238.487430
Iteration:   193   Scaled dual infeas =           127.913005
Iteration:   256   Scaled dual infeas =            33.476591
Iteration:   325   Scaled dual infeas =             6.056371
Iteration:   390   Scaled dual infeas =             5.692539
Iteration:   454   Scaled dual infeas =             3.760496
Iteration:   529   Scaled dual infeas =             2.440303
Iteration:   600   Scaled dual infeas =             2.436915
Iteration:   674   Scaled dual infeas =             2.435695
Iteration:   741   Scaled dual infeas =             2.409619
Removing shift (124).

Barrier solved model.

get_num_nodes 6
      6     8     1379.5248     4     1171.0000     1419.2047       30   21.20%             x14 U      6      5      5
get_num_nodes 7
CPXPARAM_Read_DataCheck                          1
Parallel mode: deter

Iteration:   372   Scaled dual infeas =           659.633895
Iteration:   434   Scaled dual infeas =           320.707919
Iteration:   496   Scaled dual infeas =            85.394166
Iteration:   563   Scaled dual infeas =            37.608330
Iteration:   625   Scaled dual infeas =            29.030984
Iteration:   687   Scaled dual infeas =            23.339703
Iteration:   749   Scaled dual infeas =            14.878574

Barrier solved model.

cut added
     12    12    infeasible           1209.0000     1419.2047       42   17.39%             x10 N     12     10      9
get_num_nodes 13
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 450 columns.
Reduced LP has 346 rows, 2250 columns, and 37625 nonzeros.
Presolve time = 0.02 sec. (3.57 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          5814.267573
Iteration:    62   Scaled dual infeas =          1526.628609
Ite

     16    14     1294.3084     3     1209.0000     1419.2047       48   17.39%             x15 D     16     14     12
get_num_nodes 17
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 98 columns.
Reduced LP has 190 rows, 1162 columns, and 16110 nonzeros.
Presolve time = 0.02 sec. (1.54 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          1092.000000
Iteration:    62   Scaled dual infeas =           488.826341
Iteration:   124   Scaled dual infeas =            42.480717
Iteration:   186   Scaled dual infeas =            10.544645
Iteration:   248   Scaled dual infeas =             1.656954
Iteration:   310   Scaled dual infeas =             0.097947
Iteration:   363   Dual objective     =             0.010452
Iteration:   424   Dual objective     =             0.007063
Iteration:   486   Dual objective     =             0.000012
Iteration:   548   Dual objective     

Iteration:   313   Scaled dual infeas =           204.450131
Iteration:   375   Scaled dual infeas =           189.276544
Iteration:   437   Scaled dual infeas =            83.060512
Iteration:   500   Scaled dual infeas =            45.177336
Iteration:   562   Scaled dual infeas =            12.081025
Iteration:   624   Scaled dual infeas =             4.669010

Barrier solved model.

get_num_nodes 23
     23    18     1354.7083     4     1209.0000     1417.8841       68   17.28%              x4 D     23     21      5
get_num_nodes 24
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 8 threads for concurrent optimization.
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 1152 columns.
Reduced LP has 445 rows, 3168 columns, and 65792 nonzeros.
Presolve time = 0.01 sec. (6.21 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2737.904815
Iteration:    71   Scaled dual infea

...

Reduced LP has 313 rows, 2002 columns, and 31942 nonzeros.
Presolve time = 0.02 sec. (3.03 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          3901.012491
Iteration:    62   Scaled dual infeas =           967.247270
Iteration:   124   Scaled dual infeas =           738.579461
Iteration:   186   Scaled dual infeas =           354.089058
Iteration:   248   Scaled dual infeas =            85.729277
Iteration:   310   Scaled dual infeas =            35.387550
Iteration:   372   Scaled dual infeas =             9.692886
Iteration:   434   Scaled dual infeas =             5.816174
Iteration:   496   Scaled dual infeas =             5.816174
Perturbation started.
Iteration:   505   Scaled dual infeas =             5.818954
Iteration:   567   Scaled dual infeas =             1.049152
Iteration:   629   Scaled dual infeas =             0.423064
Iteration:   691   Scaled dual infeas =             0.367025
Iteration:   753   Scaled dual

    470   116     1299.9581     4     1277.0000     1324.3715     1216    3.71%              x5 U    470    164      6
Elapsed time = 397.95 sec. (30.14 ticks, tree = 0.03 MB, solutions = 7)
get_num_nodes 471
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 98 columns.
Reduced LP has 190 rows, 1162 columns, and 16114 nonzeros.
Presolve time = 0.00 sec. (1.54 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          1029.000000
Iteration:    62   Scaled dual infeas =           310.603774
Iteration:   124   Scaled dual infeas =            17.999713
Iteration:   186   Scaled dual infeas =             6.925688
Iteration:   248   Scaled dual infeas =             3.305315
Iteration:   310   Scaled dual infeas =             1.224946
Iteration:   361   Dual objective     =             0.027319
Iteration:   422   Dual objective     =             0.009158
Iteration:   484   Dual ob

    478   113        cutoff           1277.0000     1323.8024     1237    3.67%              x9 D    478    477      7
get_num_nodes 479
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 162 columns.
Reduced LP has 235 rows, 1458 columns, and 21186 nonzeros.
Presolve time = 0.02 sec. (2.02 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2263.884601
Iteration:    62   Scaled dual infeas =           408.528828
Iteration:   124   Scaled dual infeas =           129.808911
Iteration:   186   Scaled dual infeas =            23.995908
Iteration:   249   Scaled dual infeas =            18.033007
Iteration:   311   Scaled dual infeas =             8.088575
Iteration:   373   Scaled dual infeas =             4.501786
Iteration:   435   Scaled dual infeas =             0.546457
Iteration:   482   Dual objective     =             0.046753
Iteration:   543   Dual objective   

    483   113    infeasible           1277.0000     1323.1149     1245    3.61%             x26 U    483    482     12
    484   112    infeasible           1277.0000     1323.1149     1245    3.61%             x26 D    484    482     12
get_num_nodes 485
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 242 columns.
Reduced LP has 276 rows, 1738 columns, and 26456 nonzeros.
Presolve time = 0.00 sec. (2.52 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2575.432427
Iteration:    62   Scaled dual infeas =           656.920182
Iteration:   124   Scaled dual infeas =            67.588921
Iteration:   186   Scaled dual infeas =            20.132038
Iteration:   248   Scaled dual infeas =            17.554230
Iteration:   310   Scaled dual infeas =            13.858656
Iteration:   372   Scaled dual infeas =             3.805957
Iteration:   434   Scaled dual infeas =


Root node processing (before b&c):
  Real time             =    0.08 sec. (2.39 ticks)
Sequential b&c:
  Real time             =  405.06 sec. (29.18 ticks)
                          ------------
Total (root+branch&cut) =  405.14 sec. (31.58 ticks)
MIP_optimal
Objective value =  1317.0

Column 1: Value =                 1
Column 2: Value =                 1
Column 3: Value =                 1
Column 4: Value =                 1
Column 6: Value =                 1
Column 7: Value =                 1
Column 8: Value =                 1
Column 11: Value =                 1
Column 12: Value =                 1
Column 13: Value =                 1
Column 14: Value =                 1
Column 15: Value =                 1
Column 16: Value =                 1
Column 18: Value =                 1
Column 20: Value =                 1
Column 22: Value =                 1
Column 23: Value =                 1
Column 25: Value =                 1
Column 28: Value =                 1
Column 29: Value